# Thông tin
- MSSV: 22521178
- Họ tên: Hoàng Ngọc Quân
- Bài tập: Edg Detection - BT2
- Ngày thực hiện: 14/3/2024

In [1]:
# import drive
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [7]:
import os, sys

COURSE_DIR = '/gdrive/MyDrive/CS231.O21.KHTN.1'
TASK_DIR = os.path.join(COURSE_DIR, 'Baitap', 'Baitap2')

### UT

In [4]:
class ut:
  def __init__(self): pass
  def verbose(flag, verbose, func_dir=""):
      if(flag==False): return
      print("__verbose__", func_dir)
      print(verbose)
  def over(val):
      try: ut.verbose(True, (type(val), val.shape, str(sys.getsizeof(val)) + "Bytes"))
      except Exception as e:
          ut.verbose(True, (type(val), "no-shape", str(sys.getsizeof(val)) + "Bytes"))

### Read Files

In [8]:
import numpy as np

TRAIN_DIR = os.path.join(TASK_DIR, 'images/train')
TEST_DIR = os.path.join(TASK_DIR, 'images/test')

In [9]:
os.listdir(TRAIN_DIR)

['tamgiac-1.png',
 'tamgiac-2.png',
 'tamgiac-3.png',
 'tamgiac-4.png',
 'tamgiac-5.png',
 'tron-1.png',
 'tron-2.png',
 'tron-3.png',
 'tron-4.png',
 'tron-5.png',
 'chunhat-2.png',
 'chunhat-1.png',
 'chunhat-3.png',
 'chunhat-4.png',
 'chunhat-5.png']

### Load Data

In [11]:
import cv2 as cv

def sobel_filters(img):
    Sx=np.array([[-1,0,1],[-2,0,2],[-1,0,1]],np.float32)
    Sy=np.array([[1,2,1],[0,0,0],[-1,-2,-1]],np.float32)

    Ix = cv.filter2D(img, -1, Sx)
    Iy = cv.filter2D(img, -1, Sy)

    G=np.hypot(Ix,Iy)
    G=G/G.max()*255
    theta=np.arctan2(Iy,Ix)

    return Ix,Iy,G,theta

In [12]:
def calFeartureVector(img_src):
  img_dst = img_src.copy()
  img_dst = cv.resize(img_dst, (256, 256))
  Ix, Iy, G, theta  = sobel_filters(img_dst)

  fearture = []
  for i in range(G.shape[0]):
    tmp = 0
    for j in range(G.shape[1]):
      tmp = tmp + G[i,j]
    fearture.append(tmp)

  for j in range(G.shape[1]):
    tmp = 0
    for i in range(G.shape[0]):
      tmp = tmp + G[i,j]
    fearture.append(tmp)
  return fearture

##### loop

In [26]:
def load_data(TRAIN_DIR):
  his_lis = []
  for file_name in os.listdir(TRAIN_DIR): # get lis file
    path_filename = os.path.join(TRAIN_DIR, file_name) # get file_dir
    image = cv.imread(path_filename, 0) # read img
    # transform to fea
    feat = calFeartureVector(img_src=image)
    # normalize
    # feat /= np.linalg.norm(feat)
    his_lis.append(feat)

  train = np.array(his_lis)
  train = train.reshape(train.shape[0], train.shape[1])
  ut.over(train)
  return train

In [27]:
train = load_data(TRAIN_DIR)
test = load_data(TEST_DIR)

__verbose__ 
(<class 'numpy.ndarray'>, (15, 512), '128Bytes')
__verbose__ 
(<class 'numpy.ndarray'>, (9, 512), '128Bytes')


In [29]:
train[0]

array([5.65625000e+00, 9.23046875e+00, 8.77734375e+00, 1.50917969e+01,
       1.58554688e+01, 1.98046875e+01, 1.83906250e+01, 1.69746094e+01,
       1.41425781e+01, 1.41425781e+01, 1.63066406e+01, 1.41406250e+01,
       1.64863281e+01, 1.93164062e+01, 2.05468750e+01, 4.24335938e+01,
       4.48828125e+01, 2.92382812e+01, 2.93292969e+02, 6.15744141e+02,
       6.71966797e+02, 9.08876953e+02, 1.32525391e+03, 1.52901367e+03,
       1.55835352e+03, 1.36559766e+03, 1.20866016e+03, 1.20190625e+03,
       1.20568359e+03, 1.25801367e+03, 1.25538867e+03, 1.20267578e+03,
       1.16981641e+03, 1.18908203e+03, 1.19091992e+03, 1.20180664e+03,
       1.22325000e+03, 1.25108398e+03, 1.30973047e+03, 1.39814453e+03,
       1.49693750e+03, 1.55836133e+03, 1.57768945e+03, 1.59686328e+03,
       1.62716406e+03, 1.68661133e+03, 1.75018164e+03, 1.76052734e+03,
       1.74473438e+03, 1.76168750e+03, 1.76169727e+03, 1.74516016e+03,
       1.76360352e+03, 1.75738086e+03, 1.74153906e+03, 1.73981055e+03,
      

##### predict

In [46]:
import matplotlib.pyplot as plt

def show2image(img_src, img_rst, title1, title2):
  plt.figure(figsize=(20, 20))
  # show img src
  plt.subplot(1,2,1)
  plt.title(title1)
  img_src = cv.cvtColor(img_src, cv.COLOR_BGR2RGB)
  plt.imshow(img_src, interpolation='bicubic')

  # show img result
  plt.subplot(1,2,2)
  plt.title(title2)
  img_rst = cv.cvtColor(img_rst, cv.COLOR_BGR2RGB)
  plt.imshow(img_rst, interpolation='bicubic')


In [44]:
# compute dist
from scipy.spatial.distance import cdist
dis = cdist(test, train, 'cosine')
ut.over(dis)

train_names = os.listdir(TRAIN_DIR)
test_names = os.listdir(TEST_DIR)
for i in range(test.shape[0]):
  idx = 0
  for j in range(train.shape[0]):
    if(dis[i, j] < dis[i, idx]): idx = j
  print(test_names[i], ":", train_names[idx], "__", dis[i, idx])

  path_filename = os.path.join(TRAIN_DIR, train_names[idx]) # get file_dir
  image1 = cv.imread(path_filename, 0) # read img
  path_filename = os.path.join(TEST_DIR, test_names[i]) # get file_dir
  image2 = cv.imread(path_filename, 0) # read img
  show2image(img_src=image2, img_rst=image1, title1='test', title2='train')

Output hidden; open in https://colab.research.google.com to view.

##### predict2

In [45]:
# compute dist
from scipy.spatial.distance import cdist
dis = cdist(test, train, 'euclidean')
ut.over(dis)

train_names = os.listdir(TRAIN_DIR)
test_names = os.listdir(TEST_DIR)
for i in range(test.shape[0]):
  idx = 0
  for j in range(train.shape[0]):
    if(dis[i, j] < dis[i, idx]): idx = j
  print(test_names[i], ":", train_names[idx], "__", dis[i, idx])

  path_filename = os.path.join(TRAIN_DIR, train_names[idx]) # get file_dir
  image1 = cv.imread(path_filename, 0) # read img
  path_filename = os.path.join(TEST_DIR, test_names[i]) # get file_dir
  image2 = cv.imread(path_filename, 0) # read img
  show2image(img_src=image2, img_rst=image1, title1='test', title2='train')

Output hidden; open in https://colab.research.google.com to view.